# Modelo de recomendacion

Repasemos los objetivos del modelo:

<ul><li>def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.</li>
<li>def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.</li></ul>

Para ello, vamos a usar un enfoque de recomendacion basada en contenido:


In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import Modulo as md

Intentaremos generar un sistema de recomendacion que utilice dos variables de trabajo:

<ul><li>Similitud de puntajes de usuarios.</li>
<li>Similitud de titulos.</li></ul>

In [2]:
df_puntaje = pd.read_parquet('../Datasets/parquet/Recomendacion/recomendacion.parquet')
df_puntaje.head()

,user_id,item_id,puntaje,item_name
0,76561197970982479,1250,4,Killing Floor
1,76561197970982479,22200,6,Zeno Clash
2,76561197970982479,43110,4,Metro 2033
3,js41637,251610,4,Barbie™ Dreamhouse Party™
4,js41637,227300,4,Euro Truck Simulator 2


In [3]:
unique_titles = df_puntaje['item_name'].sort_values().unique() #generamos una matriz con valores de titulos unicos
tfidf_titles = TfidfVectorizer(stop_words='english') #generamos los stop_words
#entrenamos la matriz de correlacion, nos devuelve una matrix MxM con la correlacion entre cada titulo
tfidf_matrix_titles = tfidf_titles.fit_transform(unique_titles) 
# Hacemos la correlacion coseno de la similaridad entre titulo
item_titles_sim = cosine_similarity(tfidf_matrix_titles)
# pasamos todo a un dataframe
item_titles_sim_df = pd.DataFrame(item_titles_sim, index=unique_titles, columns=unique_titles)
item_titles_sim_df

,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
! That Bastard Is Trying To Steal Our Gold !,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
0RBITALIS,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
"10,000,000",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
100% Orange Juice,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.220663,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1001 Spikes,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000
theHunter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.738334


In [4]:
#generamos una matrix con indice users, columnas items y el puntaje como valores
user_ratings_matrix = df_puntaje.pivot(index='user_id', columns='item_name', values='puntaje') 
# normalizamos segun maximos y minimos (similar a minmaxscaler)
user_ratings_matrix = user_ratings_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
#llenamos valores Nan con 0s
user_ratings_matrix.fillna(0, inplace=True)
#trasponemos la matrix
user_ratings_matrix = user_ratings_matrix.T
#eliminamos los registros con solo 0s (items sin recomendacion de ningun usuario)
user_ratings_matrix = user_ratings_matrix.loc[:, (user_ratings_matrix != 0).any(axis=0)]
user_ratings_matrix

user_id,-2SV-vuLB-Kg,-GM-Dragon,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,00000000000000000001227,00True,011111135489484797,01189958889189157253,022899,...,zombieskiler6969,zomgieee,zoozles,zp3413,zrustz16,zucchin1,zuilde,zuzuga2003,zv_odd,zyr0n1c
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [5]:
piv_sparse = sp.sparse.csr_matrix(user_ratings_matrix.values) #compresion de matriz, ya que tiene tantos 0s
piv_sparse

<3319x8298 sparse matrix of type '<class 'numpy.float64'>'
	with 32098 stored elements in Compressed Sparse Row format>

In [6]:
item_rating_similarity = cosine_similarity(piv_sparse) #hacemos una matriz de similaridad item vs item segun puntajes
user_rating_similarity = cosine_similarity(piv_sparse.T) #hacemos una matriz de similaridad de usuario vs usuario segun puntajes

In [7]:
# a dataframe
item_rating_sim_df = pd.DataFrame(item_rating_similarity, index = user_ratings_matrix.index, columns = user_ratings_matrix.index)
# a dataframe
user_rating_sim_df = pd.DataFrame(user_rating_similarity, index = user_ratings_matrix.columns, columns = user_ratings_matrix.columns)

In [8]:
item_rating_sim_df

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
user_rating_sim_df

user_id,-2SV-vuLB-Kg,-GM-Dragon,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,00000000000000000001227,00True,011111135489484797,01189958889189157253,022899,...,zombieskiler6969,zomgieee,zoozles,zp3413,zrustz16,zucchin1,zuilde,zuzuga2003,zv_odd,zyr0n1c
user_id,,,,,,,,,,,,,,,,,,,,,
-2SV-vuLB-Kg,1.000000,0.0,0.0,-0.062257,0.0,-6.002450e-01,-0.612372,0.0,0.000000,0.0,...,0.000000,0.000000,0.433013,0.0,-0.516223,0.0,0.0,0.456832,0.0,-0.063844
-GM-Dragon,0.000000,1.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.268028,0.0,...,0.000000,0.000000,0.000000,0.0,0.244023,0.0,0.0,0.000000,0.0,0.000000
-PRoSlayeR-,0.000000,0.0,1.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
-SEVEN-,-0.062257,0.0,0.0,1.000000,0.0,4.982617e-02,0.050833,0.0,0.217994,0.0,...,0.000000,0.000000,-0.035944,0.0,0.284173,0.0,0.0,-0.246490,0.0,0.005300
-_PussyDestroyer_-,0.000000,0.0,0.0,0.000000,1.0,0.000000e+00,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zucchin1,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.0,0.000000
zuilde,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.0,...,-0.049507,0.000000,0.000000,0.0,-0.006590,0.0,1.0,0.000000,0.0,0.000000
zuzuga2003,0.456832,0.0,0.0,-0.246490,0.0,-3.525573e-01,-0.373002,0.0,-0.049988,0.0,...,0.000000,0.000000,0.263752,0.0,-0.314436,0.0,0.0,1.000000,0.0,-0.106942


In [10]:
# promediamos la similaridad entre puntajes con la similaridad entre titulos de los items
item_sim_df = item_rating_sim_df.mul(0.5).add(item_titles_sim_df.mul(0.5)) 
item_sim_df

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
0RBITALIS,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
"10,000,000",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
100% Orange Juice,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.110331,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1001 Spikes,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000
theBlu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000


In [11]:
diccionario_juegos = df_puntaje[['item_id','item_name']].drop_duplicates() #diccionario de juegos
diccionario_juegos['item_id'] = diccionario_juegos['item_id'].apply(int)
md.tipo_datos(diccionario_juegos)

,Columna,Tipo_datos,%_nulos,Nulos
0,item_id,[int],0.0,0
1,item_name,[str],0.0,0


## Funcion recomendacion juego - item input

In [12]:
def recomendacion_juego(item_id):
    #verificamos que este en el diccionario
    try:
        item_name = diccionario_juegos[diccionario_juegos.item_id==item_id].item_name.values[0]
    except:
        #devuelve error
        return print(f'Sin informacion disponible para el juego {item_id}')
    count = 1
    #tomamos los juegos mas similares al juego ingresado
    #ordenando la matriz de similaridad item_item por mayor a menor en la columna del juego
    print(f'Los Juegos similares a ID juego {item_id}, nombre juego: {item_name} son:\n')
    for item in item_sim_df.sort_values(by = item_name, ascending = False).index[1:6]:
        print(f'No. {count}: ID {diccionario_juegos[diccionario_juegos.item_name==item].item_id.values[0]}, nombre {item}')
        count +=1

In [13]:
recomendacion_juego(1250) #test

Los Juegos similares a ID juego 1250, nombre juego: Killing Floor son:

No. 1: ID 232090, nombre Killing Floor 2
No. 2: ID 351570, nombre Killing Floor: Uncovered
No. 3: ID 35420, nombre Killing Floor Mod: Defence Alliance 2
No. 4: ID 521430, nombre Super Switch
No. 5: ID 13250, nombre Unreal Gold


## Funcion recomendacion juego - user input

In [14]:
def recomendacion_usuario(user_id):
    similarity_constant = 0.7 #elegimos un valor para trazar un limite entre similaridades, por debajo no se toman en cuenta
    if user_id not in user_rating_sim_df.columns: #verificamos que exista el user
        return(f'Sin informacion disponible para ese usuario {user_id}') #devolvemos error
    
    #seleccionamos los users que tienen similaridad con el user ingresado usando la constante y ordenamos de mayor a menor
    sim_users = user_rating_sim_df[user_rating_sim_df>similarity_constant].sort_values(by=user_id, ascending=False)

    best = []
    most_common = {}
    # generamos una lista con todos los items que tienen mayor similaridad que el limite con el user relacionado
    for i in sim_users:
        user_scores = user_ratings_matrix.loc[:, i]
        max_scores = user_scores[user_scores>similarity_constant]
        for j in max_scores:
            best.append(user_ratings_matrix[user_ratings_matrix.loc[:, i]==j].index.tolist())

    # contamos cuantas repeticiones tiene cada item de los users relacionados
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    #ordenamos la lista de conteo
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)

    #devolvemos la lista
    print(f'Los juegos mas recomendados segun usuarios relacionados para el usuario {user_id} son:\n')
    count = 1
    for i in sorted_list[:5]:
        print(f'No. {count}: {i[0]}')
        count +=1

    '''en conclusion, elegimos todos los juegos que tienen >0.7 de similaridad con los usuarios que tienen 
    >0.7 de similaridad con nuestro usuario ingresado y devolvemos en funcion a cantidad de veces que se repite el juego'''

In [15]:
recomendacion_usuario('-2SV-vuLB-Kg') #test

Los juegos mas recomendados segun usuarios relacionados para el usuario -2SV-vuLB-Kg son:

No. 1: Counter-Strike: Global Offensive
No. 2: Garry's Mod
No. 3: Left 4 Dead 2
No. 4: Portal 2
No. 5: Borderlands 2


## Guardado de bases de datos

In [51]:
user_rating_sim_df.to_parquet('../Datasets/parquet/Recomendacion/Final/user_rating_sim_df.parquet')
user_ratings_matrix.to_parquet('../Datasets/parquet/Recomendacion/Final/user_ratings_matrix.parquet')
diccionario_juegos.to_parquet('../Datasets/parquet/Recomendacion/Final/diccionario_juegos.parquet',index=False)
item_sim_df.to_parquet('../Datasets/parquet/Recomendacion/Final/item_sim_df.parquet') #exportacion

### Muestreo de bases de datos

Vamos a realizar un muestreo de las bases de datos porque sino el deploy es demasiado grande y supera la capacidad gratuita de render.

Para ello, vamos a atacar directamente la abse de datos mas grande que tenemos: user_rating_sim_df tomando un muestreo de usuarios de 1000 usuarios, lo mismo replicaremos sobre df_playtime para las funciones de API.

In [16]:
df_puntaje_red = pd.read_parquet('../Datasets/parquet/Recomendacion/recomendacion_red.parquet')
df_puntaje_red.head()

,user_id,item_id,puntaje,item_name
0,maplemage,211420,4,Dark Souls: Prepare to Die Edition
1,maplemage,211820,4,Starbound
2,maplemage,730,6,Counter-Strike: Global Offensive
3,maplemage,204300,6,Awesomenauts
4,thequeenpanda,42910,4,Magicka


In [17]:
user_ratings_matrix_red = df_puntaje_red.pivot(index='user_id', columns='item_name', values='puntaje')
user_ratings_matrix_red = user_ratings_matrix_red.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
user_ratings_matrix_red.fillna(0, inplace=True)
user_ratings_matrix_red = user_ratings_matrix_red.T
user_ratings_matrix_red = user_ratings_matrix_red.loc[:, (user_ratings_matrix_red != 0).any(axis=0)]
user_ratings_matrix_red

user_id,00True,12312312332123,12345678901234567890123456567890,131312,13971397,210519992016,5856565456564,64512328,723943123,76561197966979423,...,tinyrickexclamationmark,triggernyar,tsunamitad,vengerfire,walsheys,wyvaud,xjerseypsycho,yay853482185,z3b1z,zeroepix
item_name,,,,,,,,,,,,,,,,,,,,,
3D Ultra Minigolf Adventures Deluxe,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6180 the moon,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7 Days to Die,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8BitMMO,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99 Levels To Hell,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
planetarian ~the reverie of a little planet~,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
resident evil 4 / biohazard 4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
unique_titles_red = df_puntaje_red['item_name'].sort_values().unique()
tfidf_titles_red = TfidfVectorizer(stop_words='english')
tfidf_matrix_titles_red = tfidf_titles_red.fit_transform(unique_titles_red)
item_titles_sim_red = cosine_similarity(tfidf_matrix_titles_red)
item_titles_sim_df_red = pd.DataFrame(item_titles_sim_red, index=unique_titles_red, columns=unique_titles_red)
item_titles_sim_df_red

,3D Ultra Minigolf Adventures Deluxe,6180 the moon,7 Days to Die,8BitMMO,99 Levels To Hell,A Bird Story,A.V.A - Alliance of Valiant Arms,AGFPROV3 Premium,AI War: Fleet Command,APB Reloaded,...,Ziggurat,Zombie Kill of the Week - Reborn,Zoombinis,[the Sequence],how do you Do It?,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,the static speaks my name,theHunter,theHunter: Primal
3D Ultra Minigolf Adventures Deluxe,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6180 the moon,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7 Days to Die,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8BitMMO,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
99 Levels To Hell,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
planetarian ~the reverie of a little planet~,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000
theHunter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.722745


In [19]:
piv_sparse_red = sp.sparse.csr_matrix(user_ratings_matrix_red.values)
piv_sparse_red

<789x279 sparse matrix of type '<class 'numpy.float64'>'
	with 1250 stored elements in Compressed Sparse Row format>

In [20]:
item_rating_similarity_red = cosine_similarity(piv_sparse_red)
user_rating_similarity_red = cosine_similarity(piv_sparse_red.T)

In [21]:
#item similarity dataframe
item_rating_sim_df_red = pd.DataFrame(item_rating_similarity_red, index = user_ratings_matrix_red.index, columns = user_ratings_matrix_red.index)
#user similarity dataframe
user_rating_sim_df_red = pd.DataFrame(user_rating_similarity_red, index = user_ratings_matrix_red.columns, columns = user_ratings_matrix_red.columns)

In [22]:
item_sim_df_red = item_rating_sim_df_red.mul(0.5).add(item_titles_sim_df_red.mul(0.5))
item_sim_df_red

item_name,3D Ultra Minigolf Adventures Deluxe,6180 the moon,7 Days to Die,8BitMMO,99 Levels To Hell,A Bird Story,A.V.A - Alliance of Valiant Arms,AGFPROV3 Premium,AI War: Fleet Command,APB Reloaded,...,Ziggurat,Zombie Kill of the Week - Reborn,Zoombinis,[the Sequence],how do you Do It?,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,the static speaks my name,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
3D Ultra Minigolf Adventures Deluxe,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6180 the moon,0.0,1.0,0.0,0.0,0.0,-0.095435,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7 Days to Die,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8BitMMO,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
99 Levels To Hell,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,...,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
planetarian ~the reverie of a little planet~,0.0,0.0,0.0,0.0,0.0,0.490808,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000


In [23]:
diccionario_juegos_red = df_puntaje_red[['item_id','item_name']].drop_duplicates()
diccionario_juegos_red['item_id'] = diccionario_juegos_red['item_id'].apply(int)
md.tipo_datos(diccionario_juegos_red)

,Columna,Tipo_datos,%_nulos,Nulos
0,item_id,[int],0.0,0
1,item_name,[str],0.0,0


In [17]:
user_rating_sim_df_red.to_parquet('../Datasets/parquet/Recomendacion/Final/user_rating_sim_df_red.parquet')
user_ratings_matrix_red.to_parquet('../Datasets/parquet/Recomendacion/Final/user_ratings_matrix_red.parquet')
diccionario_juegos_red.to_parquet('../Datasets/parquet/Recomendacion/Final/diccionario_juegos_red.parquet',index=False)
item_sim_df_red.to_parquet('../Datasets/parquet/Recomendacion/Final/item_sim_df_red.parquet')

In [25]:
user_rating_sim_df_red.head()

user_id,00True,12312312332123,12345678901234567890123456567890,131312,13971397,210519992016,5856565456564,64512328,723943123,76561197966979423,...,tinyrickexclamationmark,triggernyar,tsunamitad,vengerfire,walsheys,wyvaud,xjerseypsycho,yay853482185,z3b1z,zeroepix
user_id,,,,,,,,,,,,,,,,,,,,,
00True,1.0,0.0,0.0,0.0,0.0,0.0,0.258199,0.0,0.000000,0.0,...,-0.632456,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0
12312312332123,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.46225,0.0,0.0
12345678901234567890123456567890,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,-0.047521,0.0,...,0.000000,0.000000,0.0,0.21777,0.000000,0.0,0.0,0.00000,0.0,0.0
131312,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,-0.033845,0.0,0.00000,-0.027634,0.0,0.0,0.00000,0.0,0.0
13971397,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0
